In [1]:
import torch
import torch.nn as nn

import json
from tqdm import tqdm

import src.models as mdls
import src.utils as utils

config = utils.get_options()
if config['use_colab']:
    utils.colab()
else:
    utils.check_if_datasets_are_downloaded()

import src.dataloaders as dataloaders
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model

Exception in thread Thread-9:
Traceback (most recent call last):
  File "c:\Users\pasin\anaconda3\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "c:\Users\pasin\anaconda3\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\pasin\anaconda3\lib\subprocess.py", line 1479, in _readerthread
    buffer.append(fh.read())
  File "c:\Users\pasin\anaconda3\lib\codecs.py", line 322, in decode
    (result, consumed) = self._buffer_decode(data, self.errors, final)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0x8e in position 0: invalid start byte
Exception in thread Thread-11:
Traceback (most recent call last):
  File "c:\Users\pasin\anaconda3\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "c:\Users\pasin\anaconda3\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\pasin\anaconda3\lib\subprocess.py", line 1479, in _readerthread
    buffer.append(fh.re

Moving kaggle file

all datasets are in place


In [2]:
img_size = config['img_size']

In [3]:
model = mdls.InspectorGadjet()
dataloader = dataloaders.dataloader

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [5]:
# Define a directory to save the checkpoints
checkpoint_dir = "checkpoints/"

# Ensure the directory exists
import os
os.makedirs(checkpoint_dir, exist_ok=True)

In [6]:
# loss_fn = nn.MSELoss()
loss_fn = mdls.combined_loss
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

model.to(device)
# loss_fn.to(device)
None

In [7]:
epochs = config['epochs']
logging = config['logging']
log_interval = config["log_wieghts_interval"]


if logging: 
    # read secrets for cometml logging
    with open('secrets.json') as secrets_file:
        secrets = json.load(secrets_file)

    # init experimenxt
    experiment = Experiment(
        api_key=secrets["api_key"],
        project_name=secrets["project_name"],
        workspace="reu-ds-club", 
    )

    hyper_params = {
        "model_name": config["model"],
        "use_colab": config['use_colab'], 
        "epochs": epochs,
        "batch_size": config['batch_size'], 
        "image_size": config['img_size'], 
    }

    experiment.log_parameters(hyper_params)

for epoch in range(epochs):
    epoch_loss = 0.0
    for sample in (pbar := tqdm(dataloader)):

        img, box = sample[0].to(device), sample[1].to(device)
        img = img.to(torch.float32)

        box = box.to(torch.float32)
        optimizer.zero_grad()
        pred = model(img)
        # loss = loss_fn(pred, box)

        loss = loss_fn(pred[0], pred[1], box)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch: {epoch}\tLoss: {epoch_loss / len(dataloader)}")

    checkpoint_filename = os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch}.pth")
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': epoch_loss,
    }, checkpoint_filename)

    if logging:
        experiment.log_metric("loss", epoch_loss, step=epoch)
    
    # logging model weights (accorging to log_interval + last epoch)
    if logging and (epoch % log_interval == 0 or epoch == epochs-1):
        torch.save(model, 'model.pth')
        experiment.log_model(name = f"model-epoch-{epoch}", file_or_folder = 'model.pth', file_name = f"model-epoch-{epoch}")
        experiment.log_asset(file_data = 'model.pth', file_name = f"model-epoch-{epoch}")
        print("save model")


if logging:
    experiment.end()

  0%|          | 0/2988 [00:00<?, ?it/s]c:\Users\pasin\anaconda3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
  0%|          | 0/2988 [00:00<?, ?it/s]


ValueError: too many values to unpack (expected 4)